# 4.0 - Model Experimentation

Goal: Evaluate multiple classifiers across engineered feature sets to select the best combination for breast cancer diagnosis.

- Primary metric: Recall 
- Secondary metrics: F1-score, Precision, Accuracy and ROC-AUC
- Feature sets: `X_processed`, `X_pca`, `X_with_ratios`, `X_poly`
- Models: Logistic Regression, Linear SVM, RBF SVM, Random Forest, XGBoost, KNN


## 1. Import and setups

In [ ]:
# Imports & setup
import os
import sys
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    RocCurveDisplay,
    PrecisionRecallDisplay,
)

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

# Project imports
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from model.data_ingestion import load_raw_data
from model.data_preprocessing import drop_unnecessary_columns, map_diagnosis_to_numerical, prepare_features_and_target

# Reproducibility
SEED = 42
np.random.seed(SEED)
warnings.filterwarnings('ignore')

# Plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)


## 2. Data loading and preprocessing

We'll recreate the feature sets by re-running the preprocessing and feature engineering pipelines that were created on the previous notebook `3.0-feature_engineering.ipynb`.

In [ ]:
# Recreate feature sets: X_processed, X_pca, X_with_ratios, X_poly, and y

# Load data
DATA_PATH = os.path.abspath(os.path.join('..', 'data', 'data.csv'))
df_raw = load_raw_data(DATA_PATH)

# Initial cleaning & mapping
df_cleaned = drop_unnecessary_columns(df_raw.copy())
df_mapped = map_diagnosis_to_numerical(df_cleaned.copy())

# Unscaled features & target (keep unscaled copy for ratios)
X_unscaled, y = prepare_features_and_target(df_mapped)

# Baseline selection (match previous notebook)
features_to_drop = [
    'fractal_dimension_se',
    'smoothness_se',
    'fractal_dimension_mean',
    'texture_se',
    'symmetry_se'
]
X_selected_unscaled = X_unscaled.drop(columns=features_to_drop)

# 1. Scaling -> X_processed
scaler = StandardScaler()
X_processed = pd.DataFrame(
    scaler.fit_transform(X_selected_unscaled),
    columns=X_selected_unscaled.columns
)
print('X_processed:', X_processed.shape)

# 2. PCA -> X_pca (n_components selected to ~95% variance in 3.0; using 8)
pca = PCA(n_components=8, random_state=SEED)
X_pca = pd.DataFrame(
    pca.fit_transform(X_processed),
    columns=[f'PC_{i+1}' for i in range(8)]
)
print('X_pca:', X_pca.shape)

# 3. Ratio features on unscaled, then scale -> X_with_ratios
X_with_ratios_unscaled = X_selected_unscaled.copy()
X_with_ratios_unscaled['shape_factor'] = (X_unscaled['perimeter_mean'] ** 2) / X_unscaled['area_mean']
X_with_ratios_unscaled['radius_growth'] = X_unscaled['radius_worst'] / X_unscaled['radius_mean']

scaler_ratios = StandardScaler()
X_with_ratios = pd.DataFrame(
    scaler_ratios.fit_transform(X_with_ratios_unscaled),
    columns=X_with_ratios_unscaled.columns
)
print('X_with_ratios:', X_with_ratios.shape)

# 4. Polynomial features: degree=2 on top-5 from EDA -> X.poly
TOP_FEATURES = [
    'concave points_worst',
    'perimeter_worst',
    'concave points_mean',
    'radius_worst',
    'perimeter_mean',
]
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(X_processed[TOP_FEATURES])
poly_feature_names = poly.get_feature_names_out(TOP_FEATURES)
X_poly_generated = pd.DataFrame(poly_features, columns=poly_feature_names)
X_poly = pd.concat([
    X_processed.drop(columns=TOP_FEATURES).reset_index(drop=True),
    X_poly_generated.reset_index(drop=True)
], axis=1)
print('X_poly:', X_poly.shape)

print('\nTarget y:', y.shape, 'Positives:', int(y.sum()), 'Negatives:', int((1 - y).sum()))


## Models and cross validation configuration

In [ ]:
# Class imbalance helper (for XGB): scale_pos_weight = neg/pos
neg = int((y == 0).sum())
pos = int((y == 1).sum())
scale_pos_weight = (neg / pos) if pos > 0 else 1.0

models = {
    'log_reg': LogisticRegression(solver='liblinear', C=1.0, class_weight='balanced', random_state=SEED),
    'linear_svc': LinearSVC(C=1.0, class_weight='balanced', random_state=SEED),
    'svc_rbf': SVC(C=1.0, kernel='rbf', gamma='scale', probability=True, random_state=SEED),
    'rf': RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_leaf=1,
        class_weight='balanced_subsample',
        n_jobs=-1,
        random_state=SEED,
    ),
    'knn': KNeighborsClassifier(n_neighbors=7, weights='distance'),
    'xgb': XGBClassifier(
        n_estimators=400,
        max_depth=3,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        reg_lambda=1.0,
        n_jobs=-1,
        eval_metric='logloss',
        random_state=SEED,
        scale_pos_weight=scale_pos_weight,
        use_label_encoder=False,
    ),
}

feature_sets = {
    'X_processed': X_processed,
    'X_pca': X_pca,
    'X_with_ratios': X_with_ratios,
    'X_poly': X_poly,
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

print('Models:', list(models.keys()))
print('Feature sets:', {k: v.shape for k, v in feature_sets.items()})

## Evaluation helper

We'll create a helper function that compute per-fold Recall, F1, Precision, Accuracy, ROC-AUC (handles predict_proba/decision_function).
Returns means, stds, and per-fold values.

In [ ]:
def get_probas_or_scores(estimator, X):
    """
    Return continuous scores for binary classification metrics.

    Prefers estimator.predict_proba(X)[:, 1] if available; falls back to
    estimator.decision_function(X); otherwise uses predicted labels (limits AUC).

    Args:
        estimator: Fitted sklearn-like estimator with predict_proba, decision_function, or predict.
        X: Feature matrix (pandas.DataFrame or numpy.ndarray).

    Returns:
        Tuple[numpy.ndarray, bool]: Scores (probabilities/decision values/labels) and a flag indicating
            whether the scores are probabilities (True) or not (False).
    """
    if hasattr(estimator, 'predict_proba'):
        proba = estimator.predict_proba(X)[:, 1]
        return proba, True
    if hasattr(estimator, 'decision_function'):
        scores = estimator.decision_function(X)
        return scores, False
    # Fallback (limited for AUC)
    preds = estimator.predict(X)
    return preds.astype(float), False


def evaluate_model_cv(estimator, X, y, cv):
    """
    Cross-validate a classifier and compute core classification metrics.

    Fits the estimator across StratifiedKFold splits and aggregates recall, F1,
    precision, accuracy, and ROC-AUC (when scores available).

    Args:
        estimator: Unfitted sklearn-like classifier (fit will occur inside each fold).
        X: Feature matrix (pandas.DataFrame).
        y: Binary target (pandas.Series or array-like).
        cv: Cross-validator (e.g., StratifiedKFold).

    Returns:
        Dict[str, Any]: For each metric, mean, std, and per-fold values:
            - 'recall_mean', 'recall_std', 'recall_folds'
            - 'f1_mean', 'f1_std', 'f1_folds'
            - 'precision_mean', 'precision_std', 'precision_folds'
            - 'accuracy_mean', 'accuracy_std', 'accuracy_folds'
            - 'roc_auc_mean', 'roc_auc_std', 'roc_auc_folds'
    """
    recalls, f1s, precs, accs, aucs = [], [], [], [], []
    
    for train_idx, test_idx in cv.split(X, y):
        X_tr, X_te = X.iloc[train_idx], X.iloc[test_idx]
        y_tr, y_te = y.iloc[train_idx], y.iloc[test_idx]

        # Fit
        model = estimator
        model.fit(X_tr, y_tr)

        # Predict labels
        y_pred = model.predict(X_te)

        # Scores for ROC-AUC
        scores, is_proba = get_probas_or_scores(model, X_te)
        try:
            auc = roc_auc_score(y_te, scores)
        except Exception:
            auc = np.nan

        # Metrics
        recalls.append(recall_score(y_te, y_pred))
        f1s.append(f1_score(y_te, y_pred))
        precs.append(precision_score(y_te, y_pred))
        accs.append(accuracy_score(y_te, y_pred))
        aucs.append(auc)

    return {
        'recall_mean': np.nanmean(recalls), 'recall_std': np.nanstd(recalls), 'recall_folds': recalls,
        'f1_mean': np.nanmean(f1s), 'f1_std': np.nanstd(f1s), 'f1_folds': f1s,
        'precision_mean': np.nanmean(precs), 'precision_std': np.nanstd(precs), 'precision_folds': precs,
        'accuracy_mean': np.nanmean(accs), 'accuracy_std': np.nanstd(accs), 'accuracy_folds': accs,
        'roc_auc_mean': np.nanmean(aucs), 'roc_auc_std': np.nanstd(aucs), 'roc_auc_folds': aucs,
    }


## Experiment loop

In [ ]:
# Run experiments across models and feature sets

results = []
raw_folds = []  # store per-fold for plots

for fs_name, X_fs in feature_sets.items():
    for model_name, estimator in models.items():
        metrics = evaluate_model_cv(estimator, X_fs, y, cv)
        row = {
            'model': model_name,
            'feature_set': fs_name,
            **{k: v for k, v in metrics.items() if not k.endswith('_folds')}
        }
        results.append(row)
        raw_folds.append({
            'model': model_name,
            'feature_set': fs_name,
            'recall_folds': metrics['recall_folds'],
            'f1_folds': metrics['f1_folds'],
        })

results_df = pd.DataFrame(results)
raw_folds_df = pd.DataFrame(raw_folds)

# Sort: Recall desc, then F1 desc
results_df = results_df.sort_values(by=['recall_mean', 'f1_mean'], ascending=[False, False]).reset_index(drop=True)

os.makedirs(os.path.abspath(os.path.join('..', 'reports', 'metrics')), exist_ok=True)
metrics_path = os.path.abspath(os.path.join('..', 'reports', 'metrics', 'model_cv_results.csv'))
results_df.to_csv(metrics_path, index=False)

print('Top results by Recall:')
display(results_df.head(10))
print('Saved metrics to: reports/metrics/model_cv_results.csv')


In [ ]:
# Visualizations: boxplots for Recall and F1 by (model, feature_set)

os.makedirs(os.path.abspath(os.path.join('..', 'reports', 'figures')), exist_ok=True)
fig_dir = os.path.abspath(os.path.join('..', 'reports', 'figures'))

# Expand raw_folds for plotting
plot_rows = []
for row in raw_folds:
    for v in row['recall_folds']:
        plot_rows.append({'metric': 'recall', 'value': v, 'model': row['model'], 'feature_set': row['feature_set']})
    for v in row['f1_folds']:
        plot_rows.append({'metric': 'f1', 'value': v, 'model': row['model'], 'feature_set': row['feature_set']})
plot_df = pd.DataFrame(plot_rows)

# Recall boxplot
plt.figure(figsize=(12, 6))
sns.boxplot(data=plot_df[plot_df['metric']=='recall'], x='feature_set', y='value', hue='model')
plt.title('Recall by Model and Feature Set (5-fold CV)')
plt.ylabel('Recall')
plt.xlabel('Feature Set')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
recall_fig_path = os.path.join(fig_dir, 'model_cv_recall_boxplot.png')
plt.tight_layout()
# plt.savefig(recall_fig_path, dpi=150)
plt.show()

# F1 boxplot
plt.figure(figsize=(12, 6))
sns.boxplot(data=plot_df[plot_df['metric']=='f1'], x='feature_set', y='value', hue='model')
plt.title('F1-score by Model and Feature Set (5-fold CV)')
plt.ylabel('F1-score')
plt.xlabel('Feature Set')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
f1_fig_path = os.path.join(fig_dir, 'model_cv_f1_boxplot.png')
plt.tight_layout()
# plt.savefig(f1_fig_path, dpi=150)
plt.show()

print('Saved plots to:', recall_fig_path, 'and', f1_fig_path)


In [ ]:
# ROC/PR curves and confusion matrices for top-2 combos (hold-out split)

# Pick top-2 by recall (then F1)
top_3 = results_df.head(3)[['model', 'feature_set']].values.tolist()

for model_name, fs_name in top_3:
    print(f'\n=== Detailed evaluation: {model_name} on {fs_name} ===')
    X_fs = feature_sets[fs_name]
    X_tr, X_te, y_tr, y_te = train_test_split(X_fs, y, test_size=0.25, stratify=y, random_state=SEED)

    est = models[model_name]
    est.fit(X_tr, y_tr)

    # Predictions and scores
    y_pred = est.predict(X_te)
    scores, is_proba = get_probas_or_scores(est, X_te)

    # Confusion matrix
    cm = confusion_matrix(y_te, y_pred)
    print('Confusion matrix:\n', cm)

    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    # ROC curve
    try:
        RocCurveDisplay.from_predictions(y_te, scores, ax=axes[0])
        axes[0].set_title(f'ROC Curve: {model_name} on {fs_name}')
    except Exception:
        axes[0].text(0.5, 0.5, 'ROC curve not available', ha='center', va='center')
        axes[0].set_axis_off()

    # PR curve
    try:
        PrecisionRecallDisplay.from_predictions(y_te, scores, ax=axes[1])
        axes[1].set_title(f'Precision-Recall Curve: {model_name} on {fs_name}')
    except Exception:
        axes[1].text(0.5, 0.5, 'PR curve not available', ha='center', va='center')
        axes[1].set_axis_off()

    plt.tight_layout()
    # Optional saves:
    # fig.savefig(os.path.join(fig_dir, f'roc_pr_{model_name}_{fs_name}.png'), dpi=150)
    plt.show()


## Conclusions & Next Steps

- The table above ranks all (model, feature set) combinations by Recall (primary), then F1.
- Use the saved CSV and figures for reporting:
  - Metrics CSV: `reports/metrics/model_cv_results.csv`
  - Figures: `reports/figures/model_cv_*.png`, plus ROC/PR images for top-2

Further improvements that could be added:
- Hyperparameter tuning (RandomizedSearchCV) for the top-3 combinations.
- Threshold tuning on the top model using PR curve to favor Recall while maintaining acceptable Precision.
- Interpretability: feature importances (tree models) or SHAP (non-PCA feature sets).

